## Imports

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

## Data preprocessing

### Loading files

In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]

train_data = pd.read_csv(
    "data/training.csv",
    header= None,
    names = cols,
    engine = "python",
    encoding = "latin1"
)

test_data = pd.read_csv(
    "data/test.csv",
    header= None,
    names = cols,
    engine = "python",
    encoding = "latin1"
)

### Preprocessing

#### Cleaning

In [ ]:
train_data.drop(["id", "date", "query", "user"],
    axis = 1,
    inplace = True)

test_data.drop(["id", "date", "query", "user"],
    axis = 1,
    inplace = True)

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
train_data_clean = [clean_tweet(tweet) for tweet in train_data.text]

In [ ]:
train_data_labels = train_data.sentiment.values
train_data_labels[train_data_labels == 4] = 1

#### Tokenization

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_data_clean, target_vocab_size = 2**16
)

train_data_inputs = [tokenizer.encode(sentence) for sentence in train_data_clean]

#### Padding

In [ ]:
MAX_LEN = max([len(sentence) for sentence in train_data_inputs])
train_data_inputs = tf.keras.preprocessing.sequence.pad_sequences(train_data_inputs,
    value = 0,
    padding = "post",
    maxlen = MAX_LEN)

### Splitting into training/testing set

In [ ]:
test_idx = np.random.randint(0, 800000, 8000)
test_idk = np.concatenate((test_idx, test_idx + 800000))

In [ ]:
test_inputs = train_data_inputs[test_idx]
test_labels = train_data_labels[test_idx]
train_inputs = np.delete(train_data_inputs, test_idx, axis = 0)
train_labels = np.delete(train_data_labels, test_idx)

### Model definition

In [ ]:
class DCNN(tf.keras.Model):

    def __init__(self, 
                vocab_size, 
                emb_dim = 128, 
                nb_filters = 50, 
                FFN_units = 512, 
                nb_classes = 2, 
                dropout_rate = 0.1,
                training = False,
                name = "dcnn"):
        super(DCNN, self).__init__(name = name)

        self.embedding = layers.Embedding(vocab_size, emb_dim)
        self.bigram = layers.Conv1D(filters = nb_filters, kernel_size = 2, padding = "valid", activation = "relu")
        self.pool_1 = layers.GlobalMaxPool1D()
        self.trigram = layers.Conv1D(filters = nb_filters, kernel_size = 3, padding = "valid", activation = "relu")
        self.pool_2 = layers.GlobalMaxPool1D()
        self.fourgram = layers.Conv1D(filters = nb_filters, kernel_size = 4, padding = "valid", activation = "relu")
        self.pool_3 = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units = FFN_units, activation = "relu")
        self.dropout = layers.Dropout(rate = dropout_rate)

        if nb_classes == 2:
            self.last_dense = layers.Dense(units = 1, activation = "sigmoid")
        else:
            self.last_dense = layers.Dense(units = nb_classes, activation = "softmax")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool_1(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool_2(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool_3(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis = -1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

### Config

In [ ]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(train_labels))

DROPOUT_Rate = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5

### Training

In [ ]:
Dcnn = DCNN(vocab_size = VOCAB_SIZE,
    emb_dim = EMB_DIM,
    nb_filters = NB_FILTERS,
    FFN_units = FFN_UNITS,
    nb_classes = NB_CLASSES,
    dropout_rate = DROPOUT_Rate)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss = "binary_crossentropy",
        optimizer = "adam",
        metrics = ["accuracy"])
else:
    Dcnn.compile(loss = "sparse_categorical_crossentropy",
        optimizer = "adam",
        metrics = ["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = "./ckpt/"

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
Dcnn.fit(train_inputs, train_labels, batch_size = BATCH_SIZE, epochs = NB_EPOCHS)
ckpt_manager.save()